## 0. Setup

In [ ]:
import os


batch_size = 2
threads = 4

# Should point to the root folder of the cropped images, see the tutorial for preparing IXI
dataset_folder = os.path.join(os.path.expanduser('~'), 'data', 'IXI', 'cropped')

## 1. Instantiate the generator

In [ ]:
from pyment.data import AsyncNiftiGenerator, NiftiDataset


dataset = NiftiDataset.from_folder(dataset_folder, target='age')

# Set up a preprocessor to normalize the voxel values
preprocessor = lambda x: x/255

generator = AsyncNiftiGenerator(dataset, preprocessor=preprocessor, 
                                batch_size=batch_size, threads=threads)

## 2. Sanity check the generator

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

batches = 0

for X, y in generator:
    fig, ax = plt.subplots(1, batch_size)
    fig.suptitle(f'Batch {batches+1}')
    
    for i in range(batch_size):
        ax[i].imshow(X[i,83,:,:])
        ax[i].axis('off')
        ax[i].set_title(round(y[i], 2))
        
    plt.show()
    
    print(f'Image batch shape: {X.shape}')
    print(f'Image voxel value range: {round(np.amin(X), 2)}-{round(np.amax(X), 2)}')
    
    batches += 1
    
    if batches > 2:
        break

## 3. Configure the model

In [ ]:
from pyment.models import RegressionSFCN


model = RegressionSFCN(weights='brain-age')

## 4. Generate predictions

In [ ]:
generator.reset()

preds, labels = model.predict(generator, return_labels=True)

In [ ]:
preds = preds.squeeze()
mae = np.mean(np.abs(preds - labels))
fig = plt.figure(figsize=(10, 10))
plt.scatter(preds, labels, c='black', alpha=0.5)
fig.suptitle('Brain age predictions')
plt.xlim([0, 100])
plt.ylim([0, 100])
plt.plot([0, 100], [0, 100], c='red')
plt.xlabel('Predictions')
plt.ylabel('Observed')
plt.show()

print(f'MAE: {mae}')